<a href="https://colab.research.google.com/github/Shaifali07/LLM_Application_Practice/blob/main/QA_System_usingrunabbles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install langchain
!pip install langchain_groq
!pip install groq
!pip install langchain_core

In [11]:
from langchain import PromptTemplate
my_template=PromptTemplate(
    input_variables=["question", "answer"],
    template="verify if the {answer} is correct for the {question}. Just reply in one word correct or incorrect"
)

In [12]:
from my_key import grq_key
import os
os.environ["GROQ_API_KEY"]=grq_key

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
llm=ChatGroq(
    model_name="llama3-70b-8192",
    temperature=0
)

evaluator_chain=my_template|llm|StrOutputParser()
evaluator_chain.invoke({"question": "What is the capital of France?", "answer": "Ahmedabad"})

'Incorrect'

Routing function

In [55]:
def routing_function(input:str):
  if(input["response"].lower()=="correct"):
    return correct_chain

  else:
    return incorrect_chain


Correct answer chain

In [15]:
correct_chain_template=PromptTemplate(
    input_variables=["Question"],
    template="User has given correct answer to the {Question}. So now ask more difficult related question. Just return a question"
)
correct_chain=correct_chain_template|llm|StrOutputParser()
correct_chain.invoke({"Question":"What is the capital of France?"})

'What is the name of the famous palace located in the Île-de-France region, about 12 miles southwest of central Paris, that was the principal royal residence of France from 1682 until the start of the French Revolution in 1789?'

Incorrect answer chain

In [42]:
incorrect_chain_template=PromptTemplate(
    input_variables=["Question","answer"],
    template="User has given incorrect answer to the {Question}. Users answer is {answer}.Explain user what is wrong with their answer. Don't give any prefix"
)
incorrect_chain=incorrect_chain_template|llm|StrOutputParser()
incorrect_chain.invoke({"Question":"What is the capital of France?","answer": "Ahmedabad"})

"Ahmedabad is not the capital of France. Ahmedabad is a city located in the state of Gujarat, India. The capital of France is actually Paris. Paris is a city located in the north-central part of France and is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It's a major cultural and economic center in Europe."

In [59]:
from  langchain_core.runnables import RunnableLambda, RunnableParallel,RunnablePassthrough
finalchain=RunnableParallel({"response":evaluator_chain, "Question": lambda x:x["question"], "answer": lambda x:x["answer"]})|RunnableParallel(reply=RunnableLambda(routing_function),input=RunnablePassthrough())
finalchain.invoke({"question":"What is the capital of France?","answer":"Paris"})

{'reply': 'What is the name of the famous palace located in the Île-de-France region, which was the principal royal residence of France from 1682 until the start of the French Revolution in 1789?',
 'input': {'response': 'Correct',
  'Question': 'What is the capital of France?',
  'answer': 'Paris'}}